# Compare-3: total claim counts are similar for consecutive timespans. If test fails, then do test Compare-9.

Description: check if total claim counts are similar for consecutive timespans. If test fails, then do test Compare-9.

Starting Author: Amy Jin (amy@careset.com)

Date: April 16th, 2018

https://docs.google.com/spreadsheets/d/1IYg01IpssJaWHo6KxO4_dSDgXtYNFy41S5cIHFLvlGQ/edit#gid=604789549

## Connect to Parenthood Server

In [1]:
# Packages import
import os
import sys
import numpy as np
import pandas as pd
from collections import Counter
import operator
import mysql.connector
import sshtunnel
import pureyaml

# Handle path
project_dir = !pwd  # dir of current script/notebook file
config_file = open(project_dir[0] + "/db.yaml");
config = pureyaml.load(config_file.read());

# Argument dictionary for sshtunnel
ssh_config = {
    'ssh_address_or_host': ('parenthood.set.care', 22),
    'ssh_username':        config['ssh_username'],
    'ssh_password':        config['ssh_password'],
    'remote_bind_address': ('127.0.0.1', 3306),
    'local_bind_address':  ('0.0.0.0', 3333),
}

# Argument dictionary for mysql.connector
mysql_config = {
    'user':     config['mysql_user'],
    'password': config['mysql_passwd'],
    'host':     config['mysql_host'],
    'database': 'patch',
    'port':     3333,
}

# Connect to Parenthood server
with sshtunnel.SSHTunnelForwarder(**ssh_config) as tunnel:
    print('SSH tunneling successful on port: {}'.format(tunnel.local_bind_port))
    connection = mysql.connector.connect(**mysql_config)
    cur = connection.cursor()
    print('MySQL server connected successfully!')

SSH tunneling successful on port: 3333
MySQL server connected successfully!


## Test Function

In [2]:
# --------------------------------------- Inputs: ---------------------------------------
# 1) db_name:                database name in server
# 2）table_name:             table name
# 3) clm_cnt:                claim count column
# --------------------------------------- Outputs: --------------------------------------
# 1) Test result:  total claim count.


def compare_3(db_name, table_name, clm_cnt):
    #table1 = str(db_name) + '.' + str(table_name)
    with sshtunnel.SSHTunnelForwarder(**ssh_config) as tunnel:
        connection = mysql.connector.connect(**mysql_config)
        cur = connection.cursor()
        
        # MySQL query to compute total claim count
        query = ('''
                SELECT SUM({col1})
                FROM {db}.{t1};
        '''.format(db = db_name, t1 = table_name, col1 = clm_cnt))

        cur.execute(query)
        
        print ("The total {} in {}.{} is:".format(clm_cnt, db_name, table_name) + '\n')        
        for row in cur.fetchall():
            for i in range(0,len(row)):
                print (str(row[i]))
            
        cur.close()
        connection.close()            

## Test Data Example

In [3]:
compare_3('_amy', 'test_data_good', 'cnt_clm_id')

The total cnt_clm_id in _amy.test_data_good is:

8513


In [4]:
compare_3('_amy', 'test_data_bad1', 'cnt_clm_id')

The total cnt_clm_id in _amy.test_data_bad1 is:

397284487540.0


In [7]:
compare_3('_amy', 'test_data_bad2', 'cnt_clm_id')

The total cnt_clm_id in _amy.test_data_bad2 is:

206158436544.0


## Internal Data Test Example

Compute total claim of HOP Teaming 2014, 2015 and 2016.

#### The total claim count of HOP Teaming 2014 is:

In [5]:
compare_3('npi_hop_RQ14', 'directed', 'transaction_count')

The total transaction_count in npi_hop_RQ14.directed is:

5163800092


#### The total claim count of HOP Teaming 2015 is:

In [4]:
compare_3('npi_hop_RQ15', 'directed', 'transaction_count')

The total transaction_count in npi_hop_RQ15.directed is:

5133369829


#### The total claim count of HOP Teaming 2016 is:

In [5]:
compare_3('npi_hop_RQ16', 'directed', 'transaction_count')

The total transaction_count in npi_hop_RQ16.directed is:

7101284629


#### The total claim count of HOP Teaming 2017 is:

In [6]:
compare_3('npi_hop_RQ17', 'directed', 'transaction_count')

The total transaction_count in npi_hop_RQ17.directed is:

13180209838
